Pandas Basics
===================================

In our overview we demonstrated how we could use pandas as a tool to help us to better understand the data that we are 
working with.  For me this is the biggest benefit that Pandas provides for use as Data Scientist, a tool to quickly
explore our dataset and through exploration to better understand the data.  

Now, I will start by saying that Pandas is not a silver bullet (as with almost all technologies).  Pandas, by itself, is used
by loading all the data into memory, which means on larger datasets Pandas starts to fail with OOM errors and incredible
long execution times.  However, if you can work with just a subset of the larger dataset it can still give you some very powerful
insight into your data.  

For this notebook I am going to focus more on having you guys learn through practice then by giving you concrete examples, to
that end I hope that you can follow along and gain a greater appreciation for pandas and the power that it provides.  

To really understand and appreciate pandas we are going to need to use it with a dataset. For this notebook we are going to 
use the [City of Austin - Traffic Count Study](https://catalog.data.gov/dataset/traffic-count-study-area/resource/820bc731-bc7c-4598-a08d-8430ad141c60).

This dataset contains the following details.  

  * Location (`24 HOUR VOLUME COUNT LOCATIONS`)
  * Northbound Total (`NB TOTAL`)
  * Southbound Total (`SB TOTAL`) 
  * Eastbound Total (`EB TOTAL`)
  * Westbound Total (`WB TOTOAL`)
  * Total Volumen (`TOTAL VOLUME`)
  * Measurement Date (`DATE`)
    
To start, we need to first import our needed modules and download the dataset we are going to work with. 

In [1]:
import csv
import math

import pandas as pd
import numpy as np

from collections import defaultdict

from ml_course.util.downloader import download_data

In [2]:
url = 'https://data.austintexas.gov/api/views/cqdh-farx/rows.csv?accessType=DOWNLOAD'
save_name = 'austin.csv'
austin_filename = download_data(url, save_name)

!head -n 5 /home/jovyan/project/ml_course/util/../../.data/austin.csv

######
File downloaded to /home/jovyan/project/ml_course/util/../../.data/austin.csv
24 HOUR VOLUME COUNT LOCATIONS,NB TOTAL,SB TOTAL,EB TOTAL, WB TOTOAL,TOTAL VOLUME,DATE
"10th St East, 1000 blk - West of Waller St",,,101,115,216,09/27/2005 12:00:00 AM
"10th St East, 1200 blk - West of Navasota St",,,103,55,158,09/27/2005 12:00:00 AM
"10th St East, 600 blk - East of Red River St",,,n/a,2240,2240,02/26/2009 12:00:00 AM
"10th St East, 700 blk - West of IH35 WSR                     1-way WB",,,n/a,1599,1599,07/06/2005 12:00:00 AM


## Getting Started

While pandas does have a lot of functionality, there are really just two main building blocks that most everything
else is built upon.  We are going to start out discussion focused on these two building blocks, how they interact
and common ways to work with them.  

The two main building blocks are the __DataFrame__ and __Series__ types.  

### DataFrame

So, what is a dataframe?  To put it in simple terms, `a dataframe is like a relational database table`.  This is
by far the easist way to see it from a Software Engineer's perspective.  The more formal answer comes from the
original source that a dataframe was modeled from, which is a data frame in the __R__ programming language.  

> A data.frame object in R has similar dimensional properties to a matrix, but it may contain categorical data
as well as numeric. Each column in the data.frame is a vector containing the variable value for a given data instance,
with each row corresponding to that instance.  

Now a Pandas DataFrame consists of three main parts, these are:

- The data
- The index (label)
- The columns (label)

Now, some of you may be asking what is the difference between a 2 dimensional numpy array and a pandas DataFrame?  There are a couple
of key things.  

1. Two-dimensional numpy arrays must all be the same type but pandas columns can be different types
2. Rows and Columns can be queried using custom string labels, not just offset indexes.  

#### Creating a Dataframe

So let's start by creating our first dataframe and seeing what it provides for us.  There are several ways that we can
create a dataframe, but for this notebook we are only going to focus on one version for simplicity. 

**NOTE:** For more details please look at the [pandas.DataFrame documentation here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)   

In [3]:
df = pd.DataFrame([
    {'id': 1, 'name': 'Frank', 'email': 'pandas@gmail.com'},
    {'id': 2, 'name': 'Ryan', 'email': 'asure@yahoo.com'},
    {'id': 3, 'name': 'George', 'email': 'old@aol.com'},
    {'id': 4, 'name': 'Jack', 'email': 'movingto@outlook.com'},
    {'id': 5, 'name': 'Luke', 'email': 'longtimeago@oldrepublic.com'}
], index=['A', 'B', 'C', 'D', 'E'])
display(df)
display(df.dtypes)

,email,id,name
A,pandas@gmail.com,1,Frank
B,asure@yahoo.com,2,Ryan
C,old@aol.com,3,George
D,movingto@outlook.com,4,Jack
E,longtimeago@oldrepublic.com,5,Luke


email    object
id        int64
name     object
dtype: object

Looking at the output above, there are a few things that we can immediately see.  

1. The output looks like a table with two axes. 
2. Our axis values have string labels, not just numeric offsets
  - Rows: A, B, C, D, E
  - Columns: email, id, name
3. Our dataframe consists of multiple types:
  - object: email, name
  - int64: id

From the datatypes describe above, we see that there is an `int64`.  This datatype is part of the numpy dtype.
Pandas is built on numpy and as such uses numpy dtypes for numeric columns.  The other type `object` is a little
bit different.  As we mentioned above, a dataframe can store not just numeric data, but also categorical data in
the form of strings.  Since strings don't have a numeric representation, they fall into the numpy catchall of
`object`.  

What is really interesting about the dtypes is that we didn't explicitly specify the type in the dataframe creation,
rather the dataframe took the time to __guess__ what the correct datatype should be.  Later on we will go over how
to fix the type when it guesses incorrectly but for now we can work with the types that have been supplied.  

Dataframes also have some very handy functions that we can use on them to view the data, some of these methods are:

- `head(n)` - Displays the first `n` rows from the dataframe
- `tail(n)` - Like head, but displays the last `n` rows instead
- `info(n)` - Overview of the dataframe including dtypes, shape and other info

Let's practice with these methods on the austin dataset.  Since we have not yet created a dataframe from that
dataset, we will first load the csv into a dataframe before running our first exercise.  

**NOTE:** While we aren't really covering it here, pandas does provide a lot of functions for working with other
data sources including: `excel`, `json`, `html` and others

In [4]:
# Note: I will use df for most of the time a dataframe is created as a simple shorthand
df = pd.read_csv(austin_filename)

In [5]:
# Exercise 1
display(df)
# 1. Using head, determine what the EB TOTAL is for the row with the index of 4

# 2. Using tail, determine what the SB TOTAL is for the row with the index of 3331

# 3. Using info, say what the data type is for NB TOTAL and EB TOTAL

# 4. Using info, determine how many entries there are in each column

,24 HOUR VOLUME COUNT LOCATIONS,NB TOTAL,SB TOTAL,EB TOTAL,WB TOTOAL,TOTAL VOLUME,DATE
0,"10th St East, 1000 blk - West of Waller St",NaN,NaN,101.0,115.0,216,09/27/2005 12:00:00 AM
1,"10th St East, 1200 blk - West of Navasota St",NaN,NaN,103.0,55.0,158,09/27/2005 12:00:00 AM
2,"10th St East, 600 blk - East of Red River St",NaN,NaN,NaN,2240.0,2240,02/26/2009 12:00:00 AM
3,"10th St East, 700 blk - West of IH35 WSR ...",NaN,NaN,NaN,1599.0,1599,07/06/2005 12:00:00 AM
4,"10th St West, 1000 blk - West of N. Lamar Blvd",NaN,NaN,373.0,288.0,661,01/24/2001 12:00:00 AM
5,"10th St West, 1300 blk - West of Lorrain St",NaN,NaN,383.0,314.0,697,06/05/2007 12:00:00 AM
6,"10th St West, 600 blk - West of Nueces St",NaN,NaN,1591.0,431.0,2022,01/11/2001 12:00:00 AM
7,"10th St West, 800 blk - West of West Ave",NaN,NaN,2163.0,925.0,3088,01/30/2003 12:00:00 AM
8,"10th St West, 900 blk - East of N. Lamar Blvd",NaN,NaN,2254.0,940.0,3194,01/30/2003 12:00:00 AM
9,"11th St East, 1100 blk - East of Waller St",NaN,NaN,4185.0,3788.0,7973,06/26/2006 12:00:00 AM


Being able to use head/tail and info are useful tools in giving us a quick view at our data.  Now that we can see some
values, we may want to retrieve specific values so that we can use the results in operations.  Because our axes have labels
we can select data using the supplied labels instead of the exact indexes (as you would have to in numpy).  

The mechanism that we will use, which is one of many, is the `loc` method.  You can find more documentation for this method
[here](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html#pandas.DataFrame.loc).  The basic
syntax for a dataframe loc method is below:

        df.loc[row, column]
        
Lets try this out by retrieving the values that we found above using the `loc` method.  

In [ ]:
# Exercise 2

# Using loc get the value for the EB TOTAL column with a row of index 4
result = df.loc[_, _]
assert result == 373.0

# Using the loc get the value for the SB TOTAL column with a row of index 3331
result = df.loc[_, _]
assert result == 903.0

The `loc` method can also work with more than just a single row and single column, namely each label can be on of:

- A single label (as in exercise 2)
- A list of array of labels
- A slice object with labels
- A boolean array

Lets try out a few of these different versions to see what information we get from each.  

In [ ]:
# Exercise 3

# Using loc create a window dataframe for rows 1000 to 1005 for the columns NB TOTAL and SB TOTAL

# Using loc retrieve all the rows for the column 'DATE' and display the first 5 row

# Using loc retrieve the first 5 rows and the columns NB TOTAL to TOTAL VOLUME (including columns in between)


So what are we doing with these different patterns?  The first example is returning a dataframe that only contains
the rows and columns that we had specified, but notice that it keeps the labels for both the rows and the columns.  

The second version is actually returning a new type that we haven't talked about yet, namely a `Series` object.  We will
discuss a series here momentarily.   

The final example is also returning a dataframe, however it is using the slice operation on strings.  This is unique
to pandas, but it understands how to work with a slice operation using the column labels, even if they are not numeric.  

**NOTE:** It is important to call out that slices in pandas do not work the same as python, where python slices are none
inclusive on the upper bound, pandas slices are inclusive.  

### Series

So in our last exercise we came across a new type from pandas, the `Series`.  A series in pandas is an objec that represents
a one-dimensional structure, much like a list.  Anytime you work with a 1D object it will be in the form of a series, 
while anytime you work with something that is 2D it will be a dataframe.  

In essence, you can consider that a dataframe is a collection of columns, where each column is a series.  _This is actually
how it is stored internally_.  

When requesting the data, if you select a single row or select a single column you will end up with a series.  It is very important
to note that series will maintain their labels whenever possible.  

Once you have access to a series, like a dataframe you can use the `loc` to retrieve an exact value, but unlike a dataframe the series
loc only accepts one dimension.  

        series.loc[label]
        
**NOTE:** Pandas heavy utilizes the notion of method chaining.  This is done so that operations can be run without the
need to intermediate temporary variables.  This means that you can run a `loc` from a series on the results of a `loc`
from a dataframe.  

        df.loc[:, 'NB TOTAL'].loc[3]

Besides just retrieving specific value from a series, there are some other useful functions that we can work with.  

- `describe()` - descriptive statiscs on data in a series
- `max()`
- `min()`
- `mean()`
- `median()`
- `mode()`
- `sum()`
- `value_counts()`

These methods are useful in giving us some basic insight into the data that we have available to us.  Let's try some of these
methods out.  

In [ ]:
# Exercise 4

# 1. First lets describe the data for the 'SB TOTAL' column

# 2. Lets use describe on the data for the `NB TOTAL` column

# 3. Get the max for the 'EB TOTAL' column and assign it to eb_total_max

#assert eb_total_max == 30033.0

# 4. Display the top 10 unique values for the `DATE` column

# 5. Get the median for the column `NB TOTAL`


As you may have noticed, the 5th part of the above exercise should have given you an error.  Namely an error about converting a string
to a number (the string being `no count`).  It seems like we have some bad data in our `NB TOTAL` column.  At this point, using only the
tools above, we would need to scan through our table to find the bad columns.  

Luckily, pandas dataframe offer more than just an easy way to view our data.  This power is part of the querying capabilities.  

## Querying

We haven't yet talked much about it, but pandas provides the ability to retrieve rows and columns from a sequence of booleans.  

        df.loc[[True, False, False, True, False, True], 'NB TOTAL':'TOTAL VOLUME']
        
This allows us to create conditions for when a row or column should be returned.  What's even more powerful is the fact that
the series object in pandas has over loaded some operators to allow a series to generate a boolean series given specific
conditions.  For example the below command will return a boolean series where the `TOTAL VOLUME` is more than 1000.  

        df.loc[:, 'TOTAL VOLUME'] > 1000
        
Besides the common overload operators, pandas provides methods that we can use to create boolean series based on a
given criteria, for example the below command will return all the rows where the value for the `SB TOTAL` column is not `NaN`.  

        pd.notna(df.loc[:, 'SB TOTAL'])
        
Other useful commands that pandas provides for querying information include.  

- `pandas.isna()`
- `pandas.isnull()`
- `pandas.notnull()`

Finally, you can combine multiple conditional queries using the `&` operator (or uses the `|` operator).  So a command
that will retrieve rows where the `SB TOTAL` is greater than 10000 but the `TOTAL VOLUME` is less than 20000 would look like
this.  

        df.loc[(df.loc[:, 'SB TOTAL'] > 10000) & (df.loc[:, 'TOTAL VOLUME'] < 20000)].head(5)

Let's start with a couple of exercises to test our knowledge of using these commands.

In [ ]:
# Exercise 5

# 1. Display the first 5 rows where the TOTAL VOLUME is greater than 10000

# 2. Display the first 5 rows where the NB TOTAL is not null

# 3. Display the rows where there is a value in both `SB TOTAL` and in `EB TOTAL`

# 4. Display the sum of 'EB TOTAL' for the rows where the `EB TOTAL` is an even number


Up until this point we have only referenced columns using the `loc` method.  Pandas does provide a few other
formats that you can use.  Each of these below commands will all return the same value.  

    df.low[:, 'DATE']
    df['DATE']
    df.DATE
    
There are also other commands that you can use besides `loc`.  

    df.loc[0, 'NB TOTAL']
    df.iloc[0, 1]
    df.at[0, 'NB TOTAL']
    df.iat[0, 1]
    
This means that the above commands could have been re-written, for example the answer for number 4 could have been written
as:

    df[df['EB TOTAL'] % 2 == 0]['EB TOTAL']
    # or
    df.loc[df['EB TOTAL'] % 2 == 0, 'EB TOTAL']
    
**NOTE:** The difference between `at` and `loc`, is that while `loc` can return a series or dataframe, `at` will only
return a scalar, this means it does not support slicing.  

**2nd NOTE:** While the different column and loc methods will return the same results, they are not the same operation
and this has to do with pandas returning a `view` or a `copy` of the data.  But for simple quering purposes they will work the same. 

Pandas dataframes also have some other useful methods for querying data values or returnings a different dataframe
to work with.  These include the following commands.  

- nsmallest(n, columns) - `df.nsmallest(5, columns='TOTAL VOLUME')`
- nlargest(n, columns) - `df.nlargest(5, columns=['TOTAL VOLUME', 'EB TOTAL'])`
- sample(n) - A random sample of `n` instances (rows) from the dataframe

In [ ]:
# Exercise 6

# 1. What is the sum of the `TOTAL VOLUME` for the 2 largest in `EB TOTAL`.  
#   Should come up with 117,269

# 2. What is the sum of the `TOTAL VOLUME` for the 3 smallest `SB TOTAL` values
#   Should come up with 132


## Cleaning the data

Some of you may have noticied or wondered (or possibly even asked a question) about why we aren't using `NB TOTAL`.  If
you recall from the very beginning, this column was of type `object` and not a numeric type, which means we can't use
or aggregate functions against that column until we fix the data.  

For this next section we are going to go through our data in an attempt to either remove or adjust results where
appropriate.  

Now there are a lot of things that we can do to our data to clean it up.  The first thing that I want to do is adjust
the names of the colums that we are using so that I can use `nb` instead of `NB TOTAL` for all of my commands.  Pandas
provides a few ways for us to do this, either by assigning the new names to the `columns` property of the dataframe
or by using the `rename` method.  

In this next exercise, rename the columns in our dataframe according to the map below.  

- `24 HOUR VOLUME COUNT LOCATIONS` -> `location`
- `NB TOTAL` -> `nb`
- `SB TOTAL` -> `sb`
- `EB TOTAL` -> `eb
- ` WB TOTOAL` -> `wb`
- `TOTAL VOLUME` -> `total`
- `DATE` -> `date`


In [ ]:
# Exercise 7

# 1. Rename all the columns according to the list above

# 2. Select the first 5 instances where the `sb` value is less than 100


Now our next cleanup task is going to be a little more involved, first we need to find all the 
data in the `nb` column that is a non-numeric, the columns that are stopping us from using our
aggregate functions.  

If we were using standard python I could compare the value to see if it can be cast to number
or not.  Since this is a common operation, pandas provides this ability for us, but using
the `str` property of a series, it will give us the functions provided by pandas for string
types, one of which is the `isnumeric()` function.  There are a few others, some of which
are listed here.  

- `str.contains()`
- `str.endswith()`
- `str.len()`
- `str.isdigit()`
- `str.isalpha()`
- `str.isspace()`

Using the above commands is done as demonstrated below.  

    df['location'].str.contains('Barton')

In [ ]:
# Exercise 8

# 1. Create a variable `na_rows` that contains a boolean series where nb is NaN
na_rows = pd.isna(df['nb'])

# 1. Create a variable `good_rows` that contains a boolean series where nb is NaN or numeric
number_rows =  (na_rows | df['nb'].str.isnumeric())

# 3. Display the rows where nb is not numeric or NaN (note you can use ~ to negate a boolean series)
df[~number_rows]

So we now have our bad rows, and it looks like we have the following errors.  

- 2 rows `during rd. construction`
- 3 rows `no count`

At this point we have a decision to make.  This data will obviously not work as we need our `nb` column
to contain numeric data, so we can do one of the following.  

1. Set the bad values to `NaN`
2. Set the bad values to 0
3. Set the bad values to the mean of the column
4. Drop the rows with the bad values

Which option we choose will larger depend upon whether we need the other information provided and if we
feel that using any of those options could drastically skew our perception of the data.  For this exercise
we are going to simply drop these rows.  This can be done by using the `drop` function.  

    df.drop(index=[row1, row2, row10])
    
Lets go ahead and run the drop command and then describe the dataframe and output the last 5 results.  

In [ ]:
# Exercise 9

# 1. Get the indexes of the rows to drop using the number_rows column above (remember to negate it) store in variable drop_rows

# 1. Run the drop command on the rows from the inverted boolean variable number_rows, save the results in dropped_df

# 2. Display the describe output for the dropped_df dataframe for column nb

# 3. Display the rows 470 to 478


For our last cleanup step, lets now fix a couple of our types.  Right now we have dropped the columns from `nb`
that were causing it to not automatically be discovered as a number so we can now convert that column.  But
the other column that is also an incorrect type is the `date` column which is currently of type `object` but
can be of type `datetime`.  

To convert an entire series, you can use the provided pandas methods:

- `pandas.to_numeric(series)`
- `pandas.to_datetime(series)`

If we want to update a column, we can do so by selecting the column using the `loc` on a dataframe.  After selecting
you need to supply a scalar (single value) or a series or sequence with the same number of instances (rows) as the
dataframe.  This is done simply by specifying the column to overwrite and the values to apply.  

    df['nb'] = new_values

For this next exercise lets create a converted series that is of the correct types.  

In [ ]:
# Exercise 10

# 1. Create a new series that is the nb series converted to numeric, store in number_nb

# 2. Create a new series that is the date series converted to a datetime, store in datetime_date

# 3. Assign the updated series to both the nb and date columns

# 4. Display the dtypes of the dataframe


So we have now cleaned up our datatypes, but we still have one thing that could cause us some problems, namely we
have `NaN` in a lot of places that if we try to run our own calculations against are going to cause issues.  So what
we want to do is to convert all the `NaN` values to 0.  In doing this we are, however, possibly lossing some information,
namely which direction a road is traveling.  

Looking over the data it seems that a road is considered traveling north to south when there is a value in either the
`nb` or the `sb` columns.  The same thing goes for `eb` and `wb`.  So what we want to do is create two more columns
on our dataframe.  These columns are going to be boolean columns and contain True when the direction traveled is either
`nb` or `sb` and `eb` or `wb`.  

Finally pandas provides a helper function that will automatically fill in all entries in a dataframe with a supplied value
when the original value in the cell is `NaN`, this command is `fillna()` on the dataframe.  

In [ ]:
# Exercise 11

# 1. Create a boolean series named ns_traffic that is true when either nb or sb is not NaN

# 2. Create a boolean series named ew_traffic that is true when either wb or eb is not NaN

# 3. Create a new column on the dataframe named `ns` and assign the ns_traffic series

# 4. Create a new column on the dataframe named `ew` and assign the ew_traffic series

# 5. Using the fillna function to set all the NaN values to 0 and save the results in zero_df

# 6. Display the first 5 rows of the zero_df dataframe


## Conclusion

At this point we now have a dataframe that has been cleaned and would be ready for further inspection
or even possible use in a machine learning model.  (__Further vectorization likely
required__).  

To make sure that we persist our changes we can store our results back to the filesystem in csv
format to be loaded later.  This is done in the following cell.  

For our next notebook lets look at the visual capabilities that are provided by pandas, especially
when used with timeseries data.  

In [ ]:
import os

save_name = os.path.join('..', '.data', 'austin_cleaned.csv')
zero_df.to_csv(save_name, index=False)